In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import numpy as np
import time
import scipy.io
import matplotlib.pyplot as plt

import torch.nn as nn
import torch

from Function.Stimulation_GPU import *
from Function.ReceptiveField_GPU import *
from Function.plot_SNN import *
from Function.SNN_JHK_v3_230317_torch import SNNModel
from Function.SNNModule import *


In [2]:
print("GPU available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)

# Enable GPU support if available
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = "cpu"

GPU available: True
CUDA version: 10.1


In [3]:
# # on mac
# device = torch.device('mps:0' if torch.backends.mps.is_available() else 'cpu')

# print (f"PyTorch version:{torch.__version__}") # 1.12.1 이상
# print(f"MPS 장치를 지원하도록 build 되었는지: {torch.backends.mps.is_built()}") # True 여야 합니다.
# print(f"MPS 장치가 사용 가능한지: {torch.backends.mps.is_available()}") # True 여야 합니다.
# !python -c 'import platform;print(platform.platform())'

In [4]:
sensor_h, sensor_w = 19, 16 #(mm)
pixel_h, pixel_w = 64, 48 # (#)
img_FPS = 60

In [5]:
num_stim = 1000

# Generate the sequence of F values
max_F = 10
F_array = np.concatenate([
    np.zeros((100,)),
    np.linspace(0, max_F, 200),
    np.full((400,), max_F),
    np.linspace(max_F, 0, 200),
    np.zeros((100,))
])

# Preallocate the stimulation_full array
stimulation_full = torch.zeros((pixel_h, pixel_w, num_stim),device = device)

# Create a color map with a range of 0 to max_F
cmap = plt.cm.get_cmap('jet')
cmap.set_under('k')

# Iterate over F_array and generate the stimuli
for i, F_i in enumerate(F_array):
    stim_dot = full_stim(F=F_i, pixel_h=pixel_h, pixel_w=pixel_w)
    stimulation_full[:, :, i] = stim_dot
    
    # if i % 200 == 0:
    #     plt.imshow(stimulation_full[:, :, i].cpu(),
    #                cmap=cmap, vmin=0, vmax=max_F)
        
    #     plt.show()
        


c:\Users\JaehunKim\anaconda3\envs\pytorch-CUDA10.1\lib\site-packages\ipykernel_launcher.py:18: MatplotlibDeprecationWarning: You are modifying the state of a globally registered colormap. In future versions, you will not be able to modify a registered colormap in-place. To remove this warning, you can make a copy of the colormap first. cmap = copy.copy(mpl.cm.get_cmap("jet"))


In [6]:
SA_rf,[SA_top_h,SA_top_v] = generate_mechanoreceptor_to_afferent_rf()
RA_rf,[RA_top_h,RA_top_v] = generate_mechanoreceptor_to_afferent_rf(kernel_w=15, kernel_h=15)

print(SA_rf.shape)
receptive_field_init = [torch.tensor([[5, -2.5], [5, -2.5], [5, -2.5], [5, -2.5], [5, -2.5]], device=device),
                        torch.tensor([[5, 5, 5, 5, 5], [-2.5, -2.5, -2.5, -2.5, -2.5]], device=device),
                        10 * torch.ones((7, 7), device=device)]

SA_rf_top, SA_rf_length = generate_afferent_to_top_rf(receptive_field_init, pixel_h=SA_top_v,
                                                                            pixel_w=SA_top_h, step_size=1)

RA_rf_top, RA_length = generate_afferent_to_top_rf(receptive_field_init, pixel_h=RA_top_v,
                                                                              pixel_w=RA_top_h, step_size=1)

print(SA_rf_top.shape)

Complete! Generated 70 receptive fields from mechanoreceptor to afferents with kernel size 10x10.
Complete! Generated 54 receptive fields from mechanoreceptor to afferents with kernel size 15x15.
torch.Size([70, 3072])
Complete! Generated 67 receptive fields from afferents to the top with kernel size 7x7.
Complete! Generated 41 receptive fields from afferents to the top with kernel size 7x7.
torch.Size([67, 70])


In [7]:
# Create Izhikevich layers
SA_layers = [
    IzhikevichLayer(0.02, 0.2, -65, 8, len(SA_rf), a_decay=1.01,device=device),
    IzhikevichLayer(0.1, 0.2, -65, 6, len(SA_rf_top), a_decay=1.01,device=device)
    ]

# Create synapses
SA_synapses = [
    Synapse(SA_rf.to(device),device = device),
    Synapse(SA_rf_top.to(device),device = device)
    ]

RA_layers = [
    IzhikevichLayer(0.02, 0.2, -65, 8, len(RA_rf), a_decay=1,device=device),
    IzhikevichLayer(0.02, 0.2, -65, 8, len(RA_rf_top), a_decay=1,device=device)
    ]

RA_synapses =[
    Synapse(RA_rf.to(device),device = device),
    Synapse(RA_rf_top.to(device),device = device)
    ]
    

layer initialized
layer initialized
layer initialized
layer initialized


In [12]:
stim = stimulation_full.to(device)
print(stim.shape[2])

SA = SNNModel(SA_layers,device)

start_time = time.time()

spike_times = SA.feedforward(stim,SA_synapses)

end_time = time.time()
vectorized_time = end_time - start_time
print(vectorized_time)

print(spike_times[0].shape)

1000
Initializing SNN Model with 2 layers with device cpu
0.33827972412109375
torch.Size([70, 999])
